# Model Training and Testing **YouTube Views Prediction**

Model Evaluation: MAE, RSME, MAPE, R^2

Techniques:
- Linear Regression
- Decision Tree
- Random Forest
- Support Vector Regressor

Regularization:
- L1
- L2 (I think this should perform better)

Hyperparameter Tuning
- Grid Search

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import os

## Load Dataset

In [ ]:
input_dir = 'data_files'

X_train = pd.read_csv(os.path.join(input_dir, 'X_train.csv'))
X_test = pd.read_csv(os.path.join(input_dir, 'X_test.csv'))
y_train = pd.read_csv(os.path.join(input_dir, 'y_train.csv'))
y_test = pd.read_csv(os.path.join(input_dir, 'y_test.csv'))